In [1]:
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import copy, time
import random
import pickle
import scipy

import mlrfit as mf
from tqdm import tqdm

import mfmodel as mfm

/Users/parshakova.tanya/opt/anaconda3/envs/routing/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.random.seed(1001)
random.seed(1001)

In [3]:
M = 5
m = 300
N = 50
rank = 40

# for _ in tqdm(range(M)):
for _ in range(M):
    # num_levels = int(np.ceil(np.log2(min(m, m))) + 1)
    hpart = mf.random_hpartition(m,  m, level_list=None, symm=True)
    ranks = mf.uniform_capped_ranks(rank, hpart)
    hat_A = mf.MLRMatrix(hpart=hpart, ranks=ranks, debug=True)
    hat_A.construct_sparse_format()
    hat_A.B[:, -1] = np.abs(hat_A.B[:, -1]) + 1e-3
    hat_A.C = hat_A.B
    # hat_A.C[:, -1] = np.abs(hat_A.C[:, -1]) + 1e-3

    part_sizes = []
    for level in range(len(hat_A.hpart['rows']['lk'])):
        part_sizes += [hat_A.hpart['rows']['lk'][level].size-1]

    F0, D0 = hat_A.B[:, :-1], np.square(hat_A.B[:, -1:])
    assert F0.shape[1] == rank-1 and np.allclose(np.concatenate([F0, np.sqrt(D0)], axis=1), hat_A.B)
    D0 = D0[:, 0]
    F_hpart = {"lk": hat_A.hpart['rows']['lk'][:-1], "pi":hat_A.hpart['rows']['pi']}

    tilde_F0 = mf.convert_compressed_to_sparse(F0, F_hpart, ranks[:-1])
    Sigma0 = mfm.perm_hat_Sigma(F0, D0, F_hpart, ranks) 
    assert np.allclose(Sigma0, tilde_F0 @ tilde_F0.T + np.diag(D0.flatten()))
    assert np.allclose(Sigma0[hat_A.pi_inv_rows, :][:, hat_A.pi_inv_cols], hat_A.matrix())

    row_selectors, si_groups, F_hpart = mfm.row_col_selections(hat_A.hpart)
    num_sparsities = row_selectors.size - 1
    lu, piv = scipy.linalg.lu_factor(Sigma0)
    Y = np.random.randn(N, m)

    F1 = mfm.EM_get_F(F0, lu, piv, Y, ranks, part_sizes, F_hpart, row_selectors, si_groups)
    tilde_F1 = mf.convert_compressed_to_sparse(F1, F_hpart, ranks[:-1])
    D_test = []
    for si in tqdm(range(num_sparsities)):
        r1, r2 = row_selectors[si: si+2]
        si_col = mfm.group_to_indices(si_groups[si], part_sizes, ranks)
        ri_F1_ciT = F1[r1:r2, :]
        # test column selection in group_to_indices
        assert np.allclose(ri_F1_ciT, tilde_F1[r1:r2, si_col].toarray())
        ri_At_ci_t, ci_B_ci, r1, r2 = mfm.EM_intermediate_matrices(tilde_F0, Y, lu, piv, ranks, si, part_sizes, si_groups, row_selectors)

        # computation of diagonal D
        M1 = (1/N) * np.diag(Y[:, r1:r2].T @ Y[:, r1:r2] - 2 * ri_F1_ciT @ ri_At_ci_t + ri_F1_ciT @ ci_B_ci @ ri_F1_ciT.T)
        M2 = (1/N) * ( np.einsum('ij,ji->i', Y[:, r1:r2].T, Y[:, r1:r2]) \
                            - 2 * np.einsum('ij,ji->i', ri_F1_ciT, ri_At_ci_t) \
                            + np.einsum('ij,jk,ki->i', ri_F1_ciT, ci_B_ci, ri_F1_ciT.T))
        D_test += [M2]
        assert np.allclose(M1, M2)
        assert np.allclose(np.sum(M1 * np.power(D0[r1:r2], -1)), 
                           np.trace(np.diag(M1) @ np.diag(np.power(D0[r1:r2], -1))))
    D_test = np.concatenate(D_test, axis=0)
    D1 = mfm.EM_get_D(F0, F1, lu, piv, Y, ranks, part_sizes, F_hpart, row_selectors, si_groups)
    assert np.allclose(D_test, D1)

del tilde_F0, tilde_F1, hat_A, F0, F1, D1, D0

num_levels=10, num_sparsities=256
(256, 9) [  0   1   3   7  15  31  63 127 255]


100%|██████████| 256/256 [00:00<00:00, 459.41it/s]


num_levels=10, num_sparsities=256
(256, 9) [  0   1   3   7  15  31  63 127 255]


100%|██████████| 256/256 [00:00<00:00, 722.57it/s]


num_levels=10, num_sparsities=256
(256, 9) [  0   1   3   7  15  31  63 127 255]


100%|██████████| 256/256 [00:00<00:00, 902.03it/s]


num_levels=10, num_sparsities=256
(256, 9) [  0   1   3   7  15  31  63 127 255]


100%|██████████| 256/256 [00:00<00:00, 466.25it/s]


num_levels=10, num_sparsities=256
(256, 9) [  0   1   3   7  15  31  63 127 255]


100%|██████████| 256/256 [00:00<00:00, 432.94it/s]


In [4]:
ranks = np.array([20, 10, 1])
rank = ranks.sum()

m = n = 100 
N = 50
L = ranks.size

hpart = mf.random_hpartition(n, n, level_list=list(range(L-1)) + [int(np.ceil(np.log2(n)))], symm=True, perm=True)
part_sizes = mfm.print_hpart_numgroups(hpart)
row_selectors, si_groups, F_hpart = mfm.row_col_selections(hpart)

Y = np.random.randn(N, m)
block_diag_perm_Y = Y[:, hpart["rows"]["pi"]]

level=0, num_groups=1, mean_size=100.0
level=1, num_groups=2, mean_size=50.0
level=2, num_groups=100, mean_size=1.0
num_levels=3, num_sparsities=2
(2, 2) [0 1]


In [5]:
def manual_sigma(F0, D0, hpart, ranks):
    assert len(hpart["rows"]['lk']) == 3
    res = np.diag(D0) + 0
    res += F0[:, :ranks[0]] @ F0[:, :ranks[0]].T
    b1, b2, b3 = hpart["rows"]['lk'][1]
    res[b1:b2, b1:b2] += F0[b1:b2, ranks[0]:ranks[:2].sum()] @ F0[b1:b2, ranks[0]:ranks[:2].sum()].T
    res[b2:b3, b2:b3] += F0[b2:b3, ranks[0]:ranks[:2].sum()] @ F0[b2:b3, ranks[0]:ranks[:2].sum()].T
    return res

In [6]:
F0, D0 = np.random.randn(m, rank-1), np.square(np.random.rand(m)) + 1e-3

F_init = F0 + 0.0
D_init = D0 + 0.0

In [7]:
loglikelihoods = [-np.inf]
for t in range(200):
    Sigma0 = mfm.perm_hat_Sigma(F0, D0, F_hpart, ranks)
    assert np.allclose(Sigma0, manual_sigma(F0, D0, hpart, ranks))

    lu, piv = scipy.linalg.lu_factor(Sigma0)
    loglikelihoods += [mfm.loglikelihood_value(Sigma0, lu, piv, block_diag_perm_Y)]

    F1 = mfm.EM_get_F(F0, lu, piv, block_diag_perm_Y, ranks, part_sizes, F_hpart, row_selectors, si_groups)

    # test computation of F1
    tilde_F0 = mf.convert_compressed_to_sparse(F0, F_hpart, ranks[:-1])
    s = tilde_F0.shape[1]
    Sigma0_inv_F0 = scipy.linalg.lu_solve((lu, piv), tilde_F0.toarray())
    Y_Sigma0_inv_F0 = block_diag_perm_Y @ Sigma0_inv_F0
    A = Y_Sigma0_inv_F0.T @ block_diag_perm_Y
    B = N * (np.eye(s) - Sigma0_inv_F0.T @ tilde_F0) + Y_Sigma0_inv_F0.T @ Y_Sigma0_inv_F0

    F1_test = np.zeros((n, ranks[:-1].sum()))
    b1, b2, b3 = hpart["rows"]['lk'][1]
    F1_test[:b2, :] = np.linalg.solve(B[:ranks[:2].sum(), :ranks[:2].sum()], A[:ranks[:2].sum(), :b2]).T
    indices = np.concatenate([np.arange(ranks[0]), ranks[1] + np.arange(ranks[0], ranks[:2].sum())], axis=0)
    F1_test[b2:, :] = np.linalg.solve(B[indices, :][:, indices], A[indices, :][:, b2:]).T

    assert np.allclose(F1, F1_test)
    D1 = mfm.EM_get_D(F0, F1, lu, piv, block_diag_perm_Y, ranks, part_sizes, F_hpart, row_selectors, si_groups)
    assert D1.min() >= -1e-8 and loglikelihoods[-2] - 1e-8 <= loglikelihoods[-1], print(loglikelihoods[-5:])
    F0, D0 = F1, D1

print("PASSED")

PASSED


In [8]:
for _ in range(10):
    A, B = np.random.randn(20, 20), np.random.randn(20, 20)
    assert np.allclose(np.einsum('ij,ji->i', A, B), np.diag(A @ B))

print("PASSED")

PASSED
